<a href="https://colab.research.google.com/github/YoonJiHwan98/TIL/blob/main/health.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn import preprocessing
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sb
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
# 한글폰트 사용 in colab
%matplotlib inline  

!apt-get update -qq
!apt-get install fonts-nanum* -qq

path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf' 
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)


In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/content/국민체육/KS_NFA_FTNESS_MESURE_ITEM_MESURE_INFO_202106.csv")
df2 = pd.read_csv("/content/drive/MyDrive/content/국민체육/국민체력측정.csv")

In [ ]:
df1.head()

In [ ]:
df1.isnull().sum()

In [ ]:
df1 = df1.drop(["ITEM_F011"],axis=1)

In [ ]:
df1 = df1.drop(["ITEM_F013","ITEM_F014","ITEM_F015","ITEM_F016","ITEM_F017",],axis = 1)

In [ ]:
df1 = df1.drop(["ITEM_F027","ITEM_F029","ITEM_F030","ITEM_F031","ITEM_F032","ITEM_F033","ITEM_F034","ITEM_F035","ITEM_F036","ITEM_F037","ITEM_F038","ITEM_F039","ITEM_F040","ITEM_F041"],axis = 1)

In [ ]:
df1.drop(["INPUT_GBN"],axis = 1)

In [ ]:
df1 = df1.drop(["ITEM_F024"],axis = 1)

In [ ]:
df1_2 = df1.drop(["ITEM_F021","ITEM_F023","ITEM_F025","ITEM_F026"],axis = 1)

In [ ]:
df3 = df1[["AGE_GBN","TEST_AGE","CERT_GBN","TEST_SEX","ITEM_F001","ITEM_F002","ITEM_F003","ITEM_F012","ITEM_F018","ITEM_F019","ITEM_F020","ITEM_F021","ITEM_F022","ITEM_F028"]]

In [ ]:
df3 = df3.dropna(axis = 0,how='any')

In [ ]:
df3

In [ ]:
df3.rename(columns = {'ITEM_F003' : '체지방율','AGE_GBN' : '나이대','TEST_AGE' : '나이','CERT_GBN' : '상장 구분','TEST_SEX' : '성별','ITEM_F001' : '키','ITEM_F002' : '체중','ITEM_F028' : '상대 악력','ITEM_F012' : '윗몸 앞으로 굽히기','ITEM_F018' : 'BMI','ITEM_F019' : '윗몸일으키기','ITEM_F020' : '왕복오래달리기','ITEM_F021' : '10M 4회 왕복달리기','ITEM_F022' : '제자리 멀리뛰기'}, inplace = True)


In [ ]:
df3 = df3.drop(columns=['상장 구분','나이대'])

In [ ]:
df3.describe()

In [ ]:
df3['성별'] = df3['성별'].map({'M':0, 'F':1})

In [ ]:
df3.head()

In [ ]:

df_scaled = StandardScaler().fit_transform(df3) 
pca = PCA(n_components=2) 
pca.fit(df_scaled) 
df_pca = pca.transform(df_scaled) 
print(df_pca.shape,df_scaled.shape)


In [ ]:
phy = df3[['나이','성별','키','체중','체지방율','BMI']]
mot = df3[['윗몸 앞으로 굽히기','윗몸일으키기','왕복오래달리기','10M 4회 왕복달리기','제자리 멀리뛰기','상대 악력']]

In [ ]:
phy_scaled = StandardScaler().fit_transform(phy)
mot_scaled = StandardScaler().fit_transform(mot) 
pca1 = PCA(n_components= 1)
pca1.fit(phy_scaled)
pca1.fit(mot_scaled)
X_pca_phy = pca1.fit_transform(phy_scaled)
X_pca_mot = pca1.fit_transform(mot_scaled)

In [ ]:
df3['Physical'] = X_pca_phy
df3['Moter Ability'] = X_pca_mot

In [ ]:
xf = X_pca_mot
yf = X_pca_phy
plt.figure()
plt.xlabel('운동 능력')
plt.ylabel('신체능력')
plt.scatter(xf, yf)
plt.show()

In [ ]:
processed_data = df3.copy()
clusters = 6
model = KMeans(init = 'k-means++', 
               n_clusters = clusters, 
               n_init = 100)
model.fit(df3[['Moter Ability','Physical']])
labels = model.labels_
df3['분류 그룹'] = labels

In [ ]:
df3

In [ ]:
plt.figure(figsize=(15,6))
# 서브플롯의 라벨링
plt.xlabel('신체 능력')
plt.ylabel('운동 능력')
# 클러스터링 그리기
plt.scatter(df3['Physical'], df3['Moter Ability'], c=df3['분류 그룹'])
plt.show()

In [ ]:
for i in range(1,7):
        plt.figure(figsize=(15,6))
       # 클러스터 생성
        estimator = KMeans(i)
        ids = estimator.fit_predict(df3[['Physical','Moter Ability']])
       # 2행 3열을 가진 서브플롯 추가 (인덱스 = i)
        plt.subplot(3, 2, i)
        plt.tight_layout()
        # 서브플롯의 라벨링
        plt.title("K value = {}".format(i))
        plt.xlabel('신체능력')
        plt.ylabel('운동능력')
        # 클러스터링 그리기
        plt.scatter(df3['Physical'], df3['Moter Ability'], c=ids)
plt.show()

In [ ]:
processed_data = df3.copy()
processed_data2 = df3.copy()
# 데이터 전처리 - 정규화를 위한 작업
scaler = preprocessing.MinMaxScaler()
processed_data[['BMI', '제자리 멀리뛰기']] = scaler.fit_transform(processed_data[['BMI', '제자리 멀리뛰기']])
# 화면(figure) 생성
plt.figure(figsize = (10, 6))

In [ ]:
for i in range(1,7):
       # 클러스터 생성
        plt.figure(figsize=(15,6))
        estimator = KMeans(i)
        ids = estimator.fit_predict(processed_data)
       # 2행 3열을 가진 서브플롯 추가 (인덱스 = i)
        plt.subplot(3, 2, i)
        plt.tight_layout()
        # 서브플롯의 라벨링
        plt.title("K value = {}".format(i))
        # 클러스터링 그리기
        plt.scatter(processed_data['BMI'], processed_data[''], c=ids)  
plt.show()

In [ ]:
group1 = df3[df3['분류 그룹'] == 1]
group2 = df3[df3['분류 그룹'] == 2]
group3 = df3[df3['분류 그룹'] == 3]
group4 = df3[df3['분류 그룹'] == 4]
group5 = df3[df3['분류 그룹'] == 5]
group6 = df3[df3['분류 그룹'] == 6]

In [ ]:
df3['왕복오래달리기'].mean()

In [ ]:
group1.sort_values('제자리 멀리뛰기')

In [ ]:
group1.loc[2501,['윗몸 앞으로 굽히기','윗몸일으키기','왕복오래달리기','10M 4회 왕복달리기','제자리 멀리뛰기','상대 악력']]

In [ ]:
X = np.arange(6)
sport_name = ['윗몸 앞으로 굽히기','윗몸일으키기','왕복오래달리기','10M 4회 왕복달리기','제자리 멀리뛰기','상대 악력'] 
plt.bar(x+0.00,group1.loc[2501,['윗몸 앞으로 굽히기','윗몸일으키기','왕복오래달리기','10M 4회 왕복달리기','제자리 멀리뛰기','상대 악력']],width = 0.25,label = '참가자')
plt.bar(x+0.25,group1[['윗몸 앞으로 굽히기','윗몸일으키기','왕복오래달리기','10M 4회 왕복달리기','제자리 멀리뛰기','상대 악력']].mean(),width = 0.25,label = '그룹 평균')
plt.bar(x+0.50,df3[['윗몸 앞으로 굽히기','윗몸일으키기','왕복오래달리기','10M 4회 왕복달리기','제자리 멀리뛰기','상대 악력']].mean(),width = 0.25,label = '전체 평균')
plt.legend()
plt.xticks(X+0.25,sport_name,rotation = 13)
plt.tight_layout()
plt.figure(figsize= (20,15))
plt.show()

In [ ]:
group1.loc[2501]
group1['그룹 순위'] = group1[['윗몸 앞으로 굽히기','윗몸일으키기','왕복오래달리기','10M 4회 왕복달리기','제자리 멀리뛰기','상대 악력']].rank(method='average', ascending=False).sum(axis = 1).rank(method='dense')
df3['전체 순위'] = df3[['윗몸 앞으로 굽히기','윗몸일으키기','왕복오래달리기','10M 4회 왕복달리기','제자리 멀리뛰기','상대 악력']].rank(method='average', ascending=False).sum(axis = 1).rank(method='dense')

In [ ]:
group1 = df3[df3['분류 그룹'] == 1]

In [ ]:
group1